# TDI Capstone Project
Name: Hongdi Zhao

### Early Childhood Development Outcome prediction using Machine Learning
- Dataset: UNICEF Multiple Indicator Cluster Survey

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.dpi'] = 144

In [ ]:
from sklearn import base
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
import pickle

from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, precision_recall_curve,average_precision_score

In [ ]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col_names].values

### Read the cleaned dataset, and check some summary statistics

In [ ]:
df = pd.read_stata('MICS6_5_ch_clean.dta')
df = df.drop(columns = ['heigh_for_age_WHO','weight_for_age_WHO','BMI_zscore_WHO'])
df = df.dropna()
print(len(df))

In [ ]:
(df['outcome_identifyletter'].value_counts()) / len(df)

In [ ]:
df.info()

### Build the machine learning model using Logistics Regression

and pickle the pipeline

In [ ]:
df = pd.read_stata('MICS6_5_ch_clean.dta')
df = df.drop(columns = ['heigh_for_age_WHO','weight_for_age_WHO','BMI_zscore_WHO',
                       'outcome','outcome_read4words','outcome_recog_symbol',
                            'outcome_pickupthing_2finger','outcome_dothing_independent',
                            'outcome_getawother','year','has_hometoy','area','melevel'])
df = df.dropna()
# here is the logistics regression
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):

    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col_names].values
    
y = df['outcome_identifyletter'].values
cat_f = ['country','mom_edu','windex5']
num_f = ['earlyc_edu','age','has_shoptoy','num_books']

X_prep = df.drop(columns = ['outcome_identifyletter','sex'])

X_train, X_test, y_train, y_test = train_test_split(X_prep, y, test_size=0.35, random_state=42)

CST = ColumnSelectTransformer(num_f)
num_pipeline = Pipeline([('attribs_adder', CST),
        ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([("num", num_pipeline, num_f),
        ("cat_2", OneHotEncoder(sparse=False), cat_f)])

pipe = Pipeline([("Column Transform", full_pipeline),
                 ("Logistics Regression", LogisticRegression())])

pipe.fit(X_train,y_train)
print("Training Accuracy", pipe.score(X_test,y_test))
print((df['outcome_identifyletter'].value_counts()) / len(df))

In [ ]:
pickle.dump(pipe, open('ECD_logistics_identifyletters','wb'))

### Check the coefficients

In [ ]:
a = full_pipeline.named_transformers_['cat_2'].categories_
cat_f1_1 = []
for i in a:
    for j in i:
        cat_f1_1.append(j)

#coef = rf.feature_importances_
coef = pipe.named_steps['Logistics Regression'].coef_
feat = num_f + cat_f1_1

for feature, coeffcient in (sorted(zip(coef[0,:],feat), reverse=True)):
    print(feature, coeffcient)

## Check the model
- Confusion matrix
- Cross validation score
- Precision and recall curve
- ROC curve
- Calibration graph

In [ ]:
# Check the above model Confusion Matrix with sklearn
confusion_matrix(y_test, pipe.predict(X_test))

In [ ]:
# cross validation score
# y_train_pred = cross_val_score(pipe, X_train, y_train, cv=3, scoring="accuracy")
# y_train_pred

from sklearn.model_selection import cross_val_predict
y_scores = cross_val_predict(pipe, X_train, y_train, cv=3,
                             method="decision_function")

In [ ]:
def plot_precision_vs_recall(precisions, recalls):
    plt.plot(recalls, precisions, "b-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.axis([0, 1, 0, 1])
    plt.grid(True)

plt.figure(figsize=(8, 6))
plot_precision_vs_recall(precisions, recalls)

plt.savefig("precision_vs_recall_plot.png")
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_train, y_scores)
def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--') # dashed diagonal
    plt.axis([0, 1, 0, 1])                                    
    plt.xlabel('False Positive Rate (Fall-Out)', fontsize=16) 
    plt.ylabel('True Positive Rate (Recall)', fontsize=16)   
    plt.grid(True)                                            

plt.figure(figsize=(8, 6))                         
plot_roc_curve(fpr, tpr)
plt.savefig("roc_curve_plot.png")                        
plt.show()

In [ ]:
roc_auc_score(y_train, pipe.predict(X_train))

In [ ]:
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
from sklearn.calibration import CalibratedClassifierCV, calibration_curve

y = df['outcome_identifyletter'].values
cat_f = ['country']
num_f = ['earlyc_edu','age','melevel','windex5',
         'has_shoptoy','num_books']

X_prep = df.drop(columns = ['outcome_identifyletter','sex'])
X = full_pipeline.fit_transform(X_prep)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35,
                                                    random_state=42)

def plot_calibration_curve(est, name, fig_index):
    """Plot calibration curve for est w/o and with calibration. """
    # Calibrated with isotonic calibration
    isotonic = CalibratedClassifierCV(est, cv=2, method='isotonic')

    # Calibrated with sigmoid calibration
    sigmoid = CalibratedClassifierCV(est, cv=2, method='sigmoid')

    # Logistic regression with no calibration as baseline
    lr = LogisticRegression(C=1., solver='lbfgs')
    
    # Logistic regression with no calibration as baseline
    rf = RandomForestClassifier()

    fig = plt.figure(fig_index, figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    ax2 = plt.subplot2grid((3, 1), (2, 0))

    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")
    for clf, name in [(lr, 'Logistic'),
                      (rf, 'Random Forest'),
                      (est, name),
                      (isotonic, name + ' + Isotonic'),
                      (sigmoid, name + ' + Sigmoid')]:
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_test)[:, 1]
        else:  # use decision function
            prob_pos = clf.decision_function(X_test)
            prob_pos = \
                (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())

        clf_score = brier_score_loss(y_test, prob_pos, pos_label=y.max())
        print("%s:" % name)
        print("\tBrier: %1.3f" % (clf_score))
        print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))
        print("\tRecall: %1.3f" % recall_score(y_test, y_pred))
        print("\tF1: %1.3f\n" % f1_score(y_test, y_pred))

        fraction_of_positives, mean_predicted_value = \
            calibration_curve(y_test, prob_pos, n_bins=10)

        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
                 label="%s (%1.3f)" % (name, clf_score))

        ax2.hist(prob_pos, range=(0, 1), bins=10, label=name,
                 histtype="step", lw=2)

    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    ax2.set_xlabel("Mean predicted value")
    ax2.set_ylabel("Count")
    ax2.legend(loc="upper center", ncol=2)

    plt.tight_layout()

# Plot calibration curve for Gaussian Naive Bayes
plot_calibration_curve(GaussianNB(), "Naive Bayes", 1)
plt.savefig('calibration_GaussianNB.png',dpi=199)


# Plot calibration curve for Linear SVC
plot_calibration_curve(LinearSVC(max_iter=10000), "SVC", 2)
plt.savefig('calibration_LinearSVC.png',dpi=199)

# Plot calibration curve for Random Forest SVC
plot_calibration_curve(RandomForestClassifier(),"RF", 3)
plt.savefig('calibration_RandomForest.png',dpi=199)


plt.show()

# The above model is everything for logistics regression


#### Other models:
#### I also tried other models to see what would be the prediction accuracy
You can find the code as follows

In [ ]:
df = pd.read_stata('MICS6_5_ch_clean.dta')
df = df.drop(columns = ['heigh_for_age_WHO','weight_for_age_WHO','BMI_zscore_WHO'])
df = df.dropna()

y = df['outcome_identifyletter'].values
cat_f = ['area','sex','country']
num_f = ['earlyc_edu','age','melevel','windex5','has_hometoy',
         'has_shoptoy','year','num_books']

class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.col_names].values
        
CST = ColumnSelectTransformer(num_f)
num_pipeline = Pipeline([('attribs_adder', CST),
        ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([("num", num_pipeline, num_f), 
                                   ("cat_2", OneHotEncoder(sparse=False), cat_f)])

X_prep = df.drop(columns = ['outcome','outcome_identifyletter',
                            'outcome_read4words','outcome_recog_symbol',
                            'outcome_pickupthing_2finger','outcome_dothing_independent',
                            'outcome_getawother'])


X = full_pipeline.fit_transform(X_prep)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")
kfold = model_selection.KFold(n_splits=10, random_state=42)  # K-Folds cross-validator
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
#models.append(('SVM', SVC(kernel='rbf')))
models.append(('RF', RandomForestClassifier()))

for name, model in models:
    model.fit(X_train,y_train)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
    
    y_pred = model.predict(X_test)
    acc_score = accuracy_score(y_test, y_pred)
    msg = "%s: %f %f (%f)" % (name, cv_results.mean(),acc_score, cv_results.std())
    print(msg)